In [11]:
intervalMinutes = 5
datasetName = 'sample_data'
binsPerDay = ( 60/ intervalMinutes) * 24
binsPerWeek = binsPerDay * 7
wmaWindowSize = (2 * (60 / intervalMinutes)) + 1
numWeeks = 4
daysPerWeek = 7

In [12]:
library(jsonlite)
library(readr)

pad <- function(data, leftPadSize, rightPadSize){
    newLength = leftPadSize + length(data) + rightPadSize;
    result <- c(newLength)
    for (i in 1:newLength) { 
        result[i + leftPadSize] <- data[i]
    }
    result
}

smoothWma <- function(data, width) {
    halfWidth <- width / 2
    height <- halfWidth + 1
    totalWeight <- height * height
    newLength <- length(data) - width + 1
    result <- c(newLength) 
    
    for (i in 1:newLength) { 
        sum = 0.0
        jCenter <- i + halfWidth;
        for (j in jCenter - halfWidth:jCenter + halfWidth) { 
            weight <- height - abs(jCenter - j)
            sum <- sum + weight * data[j]
        }
        result[i] <- sum / totalWeight
    }
    result
}

plus <- function(data, amount){
    result <- c(length(data))
    for (i in 1:length(data)) { 
        result[i] <- data[i] + amount
      }
   result
}

toLogDailyAndRemainder <- function(logData, logTrend, logWeekly) {
    result <- c(length(logData))
    for (i in 1:length(logData)) { 
        result[i] <- logData[i] - (logTrend[i] + logWeekly[i])
    }
    result
}

toLogDowSubcycles <- function(data) {
    numWeeks <- numWeeks
    binsPerDay <- binsPerDay
    binsPerWeek <- binsPerWeek
    result <- matrix(nrow = daysPerWeek, ncol= numWeeks * binsPerDay, dimnames = list(c(), c()))
    for (i in 1:daysPerWeek) { 
        dataOffset <- (i-1) * binsPerDay
        for (j in 1:numWeeks) { 
            dataStart <- ((j-1) * binsPerWeek) + dataOffset
            resultStart <- (j-1) * binsPerDay
            for (k in 1:binsPerDay) { 
                result[i, (resultStart + k)] <- data[(dataStart + k)]
            }
        }
    }
    result
}

toLogDaily <- function(logDowSubcycles) {
    np <- binsPerDay
    numWeeks <- numWeeks
    numBinsPerDay <- binsPerDay
    numBinsPerWeek <- binsPerWeek
    numBinsPerDow <- numWeeks * numBinsPerDay
    logDowSeasonals <- matrix(nrow = daysPerWeek, ncol= numWeeks * binsPerDay, dimnames = list(c(), c()))    
    for (i in 1:daysPerWeek) { 
        logDowSeasonals[i,] = decompose(logDowSubcycles[i,], numBinsPerDay, numBinsPerDow)$time.series[, 'seasonal']
    }
    resultSize <- daysPerWeek * numBinsPerDow
    result <- c(resultSize)
    for (i in 1:numWeeks) { 
        for (j in 1:daysPerWeek) { 
            for (k in 1:numBinsPerDay) { 
                resultIndex <- ((i-1) * numBinsPerWeek) + ((j-1) * numBinsPerDay) + k
                subcycleIndex <- ((i-1) * numBinsPerDay) + k
                result[resultIndex] <- logDowSeasonals[j, subcycleIndex]
            }
        }
    } 
    result
}


toTrend <- function(logTrend) {
    trend <- plus(exp(logTrend), -1)
    padSize <- trunc(wmaWindowSize / 2)
    padded <- pad(trend, padSize, padSize)
    for (i in 1:padSize) {
        padded[i] <- trend[1]
        padded[(length(padded) - padSize)+ i] <- trend[length(trend) - 1]
      }
    smoothWma(padded, wmaWindowSize)
}

toSeasonal <- function(logSeasonal) {
    seasonal <- plus(exp(logSeasonal), -1)
    padSize <- trunc(wmaWindowSize / 2)
    padded <- pad(seasonal, padSize, padSize)
    for (i in 1:padSize) {
        padded[i] <- seasonal[(length(seasonal) - padSize) + i]
    }
    for (i in 1:padSize) {
        padded[length(padded) - padSize + i] <- seasonal[i]
    }
    smoothWma(padded, wmaWindowSize)
}


fitS1Model <- function(values) {
    fit <- decompose(log(plus(values, 1)), binsPerWeek, length(values))
    trend <- toTrend(fit$time.series[, 'trend'])
    seasonal <- toSeasonal(fit$time.series[, 'seasonal'])
    result <- list(trend=trend,seasonal=seasonal)
    result
}

fitS2Model <- function(values) {
    binsPerDay <- binsPerDay
    binsPerWeek <- binsPerWeek
    logData <- log(plus(values, 1))
    stlResult1 <- decompose(logData, binsPerDay, length(values))
    stlResult2 <- decompose(stlResult1$time.series[, 'trend'], binsPerWeek, length(values))
    logTrend <- stlResult2$time.series[, 'trend']
    logWeekly <- stlResult2$time.series[, 'seasonal']
    logDailyAndRemainder <- toLogDailyAndRemainder(logData, logTrend, logWeekly)
    logDowSubcycles <- toLogDowSubcycles(logDailyAndRemainder)
    logDaily <- toLogDaily(logDowSubcycles)
    trend <- toTrend(logTrend)
    weekly <- toSeasonal(logWeekly)
    daily <- toSeasonal(logDaily)
    model <- list(trend = trend, daily = daily, weekly = weekly)
    write.model(model, "aquila-model")
}

estimate <- function(data) {
    if (binsPerDay <= 1) {
        fitS1Model(data)
    }
    else {
        fitS2Model(data)
    }
}

decompose <- function(data, np, n) {
    data.ts <- ts(data, frequency = np)
    fit <- stl(data.ts, s.window = 10 * n + 1, robust = TRUE)
    fit
}

read.data <- function(name) {
    path <- paste(name, ".csv", sep="")
    read.csv(path, header=TRUE)
}


write.model <- function(model, name) {
    dir.create(file.path("out/models"), recursive=TRUE, showWarnings=FALSE)
    path <- paste("out/models/", name, ".json", sep="")
    json <- toJSON(model)
    write_json(model, path)
}

path <- paste("data/", datasetName, sep="")
data <- read.data(path)
current <- data[,2]
midpointModel <- estimate(current)

